In [ ]:
from ultralytics import YOLO
import os, time, csv, glob
import torch
import gc

# ================== CONFIG ==================
data_yaml = r"F:\skills-copilot-codespaces-vscode\thesis\rsuddataset\rsud20k\images\data_fixed.yaml"
epochs = 50                      # Full training
output_dir = "results_yolo"
detect_dir = os.path.join(output_dir, "detections")
os.makedirs(output_dir, exist_ok=True)
os.makedirs(detect_dir, exist_ok=True)

# All YOLO models
families = ["v8", "v10", "v11"]
variants = ["n", "s", "m", "l", "x"]

csv_path = os.path.join(output_dir, "yolo_train_eval_results.csv")

# ================== CHECK COMPLETED MODELS ==================
completed_models = set()
if os.path.exists(csv_path):
    try:
        import pandas as pd
        df = pd.read_csv(csv_path)
        completed_models = set(df[df['mAP50'] != 'ERROR']['Model'].tolist())
        print(f"✅ Found {len(completed_models)} completed models: {completed_models}")
    except:
        pass
else:
    # Initialize CSV if doesn't exist
    with open(csv_path, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Model", "mAP50", "mAP50-95", "Precision", "Recall", "Accuracy", "FPS", "TrainTime(s)", "ValTime(s)"])

# ================== RESUME YOLO10X FROM CHECKPOINT ==================
print("\n" + "=" * 60)
print("? RESUMING TRAINING - Starting from interrupted model")
print("=" * 60)

# Check if yolo10x has a checkpoint to resume from
yolo10x_checkpoint = os.path.join(output_dir, "yolov10x", "weights", "last.pt")
if os.path.exists(yolo10x_checkpoint) and "yolov10x" not in completed_models:
    print(f"\n{'='*60}")
    print(f"🔄 RESUMING yolov10x from epoch 23 checkpoint")
    print(f"{'='*60}")
    
    try:
        torch.cuda.empty_cache()
        gc.collect()
        
        model = YOLO(yolo10x_checkpoint)  # Load from checkpoint
        start_train = time.time()
        
        print(f"📊 Resuming from checkpoint: {yolo10x_checkpoint}")
        print(f"📊 Batch: 4, Image: 512x512")
        
        model.train(
            data=data_yaml,
            epochs=epochs,        # Will continue from epoch 23 to 50
            imgsz=512,
            device=0,
            project=output_dir,
            name="yolov10x",
            batch=4,
            exist_ok=True,
            cache=False,
            workers=0,
            amp=True,
            plots=False,
            save_period=5,        # Save checkpoint every 5 epochs
            val=True,
            verbose=True,
            patience=20,
            resume=True           # RESUME from checkpoint!
        )
        train_time = time.time() - start_train

        # Validate
        print(f"\n🔍 Evaluating yolov10x...")
        start_val = time.time()
        metrics = model.val(data=data_yaml, imgsz=512, split="val", device=0, save=False)
        val_time = time.time() - start_val

        mAP50 = metrics.box.map50
        mAP5095 = metrics.box.map
        precision = metrics.box.mp
        recall = metrics.box.mr
        fps = 1000 / metrics.speed["inference"] if metrics.speed and "inference" in metrics.speed else 0

        TP = recall * 100
        FP = (1 - precision) * 100
        FN = (1 - recall) * 100
        TN = 100 - (TP + FP + FN)
        accuracy = max(0, min(1, (TP + TN) / 100))

        print(f"✅ yolov10x | mAP50={mAP50:.3f} | Acc={accuracy:.3f} | FPS={fps:.1f}")

        # Save to CSV
        with open(csv_path, "a", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(["yolov10x", mAP50, mAP5095, precision, recall, accuracy, fps, train_time, val_time])
        
        completed_models.add("yolov10x")
        print(f"🎉 yolov10x COMPLETE!")
        
    except Exception as e:
        print(f"❌ ERROR resuming yolov10x: {str(e)}")
        import traceback
        traceback.print_exc()
    
    finally:
        try:
            del model
        except:
            pass
        torch.cuda.empty_cache()
        gc.collect()

# ================== CONTINUE WITH REMAINING MODELS ==================
print(f"\n{'='*60}")
print(f"🚀 CONTINUING WITH REMAINING YOLO11 MODELS")
print(f"{'='*60}")

for family in families:
    for variant in variants:
        model_name = f"yolo{family}{variant}"
        
        # Skip already completed models
        if model_name in completed_models:
            print(f"\n⏩ Skipping {model_name} (already completed)")
            continue
        
        # Skip models before yolo10x (already done)
        if family == "v8" or (family == "v10" and variant != "x"):
            print(f"\n⏩ Skipping {model_name} (already trained)")
            continue
        
        weights = f"{model_name}.pt"

        print(f"\n{'='*60}")
        print(f"Training {model_name} for {epochs} epochs")
        print(f"{'='*60}")
        
        torch.cuda.empty_cache()
        gc.collect()
        
        model = YOLO(weights)
        start_train = time.time()
        
        # Batch size settings
        if variant in ["x", "l"]:
            batch_size = 4
            img_size = 512
        elif variant == "m":
            batch_size = 8
            img_size = 640
        else:
            batch_size = 16
            img_size = 640
        
        print(f"📊 Batch: {batch_size}, Image: {img_size}x{img_size}")
        
        try:
            model.train(
                data=data_yaml,
                epochs=epochs,
                imgsz=img_size,
                device=0,
                project=output_dir,
                name=model_name,
                batch=batch_size,
                exist_ok=True,
                cache=False,
                workers=0,
                amp=True,
                plots=False,
                save_period=5,     # Save checkpoint every 5 epochs
                val=True,
                verbose=True,
                patience=20
            )
            train_time = time.time() - start_train

            # Validate
            print(f"\n🔍 Evaluating {model_name}...")
            start_val = time.time()
            metrics = model.val(data=data_yaml, imgsz=img_size, split="val", device=0, save=False)
            val_time = time.time() - start_val

            mAP50 = metrics.box.map50
            mAP5095 = metrics.box.map
            precision = metrics.box.mp
            recall = metrics.box.mr
            fps = 1000 / metrics.speed["inference"] if metrics.speed and "inference" in metrics.speed else 0

            TP = recall * 100
            FP = (1 - precision) * 100
            FN = (1 - recall) * 100
            TN = 100 - (TP + FP + FN)
            accuracy = max(0, min(1, (TP + TN) / 100))

            print(f"✅ {model_name} | mAP50={mAP50:.3f} | Acc={accuracy:.3f} | FPS={fps:.1f}")

            # Save to CSV
            with open(csv_path, "a", newline="") as f:
                writer = csv.writer(f)
                writer.writerow([model_name, mAP50, mAP5095, precision, recall, accuracy, fps, train_time, val_time])
            
            print(f"🎉 {model_name} COMPLETE!")
            
        except Exception as e:
            print(f"❌ ERROR training {model_name}: {str(e)}")
            import traceback
            traceback.print_exc()
            
            with open(csv_path, "a", newline="") as f:
                writer = csv.writer(f)
                writer.writerow([model_name, "ERROR", "ERROR", "ERROR", "ERROR", "ERROR", "ERROR", 0, 0])
        
        finally:
            try:
                del model
            except:
                pass
            torch.cuda.empty_cache()
            gc.collect()
            
            if torch.cuda.is_available():
                mem_alloc = torch.cuda.memory_allocated(0) / 1024**3
                mem_reserved = torch.cuda.memory_reserved(0) / 1024**3
                print(f"🧹 GPU Memory: {mem_alloc:.2f}GB allocated, {mem_reserved:.2f}GB reserved\n")

# ================== FINAL SUMMARY ==================
print("\n" + "=" * 60)
print("ALL TRAINING COMPLETE!")
print("=" * 60)
print(f"📄 Results CSV: {csv_path}")
print(f"📦 Model Weights: {output_dir}/*/weights/best.pt")

print("\n📊 RESULTS SUMMARY:")
try:
    import pandas as pd
    df = pd.read_csv(csv_path)
    df_clean = df[df['mAP50'] != 'ERROR'].copy()
    df_clean['mAP50'] = pd.to_numeric(df_clean['mAP50'])
    print(df_clean.to_string(index=False))
    
    if len(df_clean) > 0:
        best_idx = df_clean['mAP50'].idxmax()
        best_model = df_clean.loc[best_idx, 'Model']
        best_map = df_clean.loc[best_idx, 'mAP50']
        print(f"\n🏆 BEST MODEL: {best_model} with mAP50 = {best_map:.3f}")
        
        print(f"\n📈 PROGRESS: {len(df_clean)}/15 models completed ({len(df_clean)/15*100:.1f}%)")
except Exception as e:
    print(f"Error displaying results: {e}")


In [7]:
from ultralytics import YOLO
import os, time, csv
import torch
import gc
from pathlib import Path

# ================== TRAIN ALL 15 YOLO MODELS ==================
print("=" * 60)
print("🚀 Training ALL 15 YOLO Models (v8, v10, v11)")
print("=" * 60)

data_yaml = r"F:\skills-copilot-codespaces-vscode\thesis\rsuddataset\rsud20k\images\data_fixed.yaml"
epochs = 50
output_dir = "results_yolo"
csv_path = os.path.join(output_dir, "yolo_train_eval_results.csv")

# Base directory for model weights (same directory as notebook)
weights_dir = r"F:\skills-copilot-codespaces-vscode\thesis\all code\weights"

# ALL 15 YOLO models to train
all_models = []
for family in ["v8", "v10", "v11"]:
    for variant in ["n", "s", "m", "l", "x"]:
        all_models.append(f"yolo{family}{variant}")

print(f"📋 Total models to train: {len(all_models)}")
print(f"   Models: {', '.join(all_models)}")

# ================== CHECK COMPLETED MODELS ==================
completed_models = set()
if os.path.exists(csv_path):
    try:
        import pandas as pd
        df = pd.read_csv(csv_path)
        completed_models = set(df[df['mAP50'] != 'ERROR']['Model'].tolist())
        print(f"\n✅ Found {len(completed_models)} completed models: {completed_models}")
    except Exception as e:
        print(f"⚠️ Could not read CSV (may be open in Excel): {e}")
else:
    # Initialize CSV if doesn't exist
    print(f"\n📝 Creating new CSV file...")
    os.makedirs(output_dir, exist_ok=True)
    with open(csv_path, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Model", "mAP50", "mAP50-95", "Precision", "Recall", "Accuracy", "FPS", "TrainTime(s)", "ValTime(s)"])

# Calculate remaining models
remaining_models = [m for m in all_models if m not in completed_models]
print(f"\n📊 Progress: {len(completed_models)}/{len(all_models)} completed, {len(remaining_models)} remaining")
if remaining_models:
    print(f"   Remaining: {', '.join(remaining_models)}")

print(f"\n⚡ Starting training loop...\n")

# ================== TRAIN EACH MODEL ==================
for idx, model_name in enumerate(all_models, 1):
    # Skip if already completed
    if model_name in completed_models:
        print(f"\n[{idx}/{len(all_models)}] ⏩ Skipping {model_name} - already completed!")
        continue
    
    print(f"\n{'='*60}")
    print(f"[{idx}/{len(all_models)}] 🚀 Training {model_name}")
    print(f"{'='*60}")
    
    torch.cuda.empty_cache()
    gc.collect()
    
    # Check for checkpoint
    checkpoint = os.path.join(output_dir, model_name, "weights", "last.pt")
    resume = False
    
    if os.path.exists(checkpoint):
        # Check if checkpoint is finished
        try:
            ckpt = torch.load(checkpoint, map_location='cpu')
            last_epoch = ckpt.get('epoch', -1)
            if last_epoch >= epochs - 1:
                print(f"✅ {model_name} already trained to {last_epoch + 1} epochs")
                
                # Run validation to get metrics
                print(f"🔍 Running validation to record metrics...")
                model = YOLO(os.path.join(output_dir, model_name, "weights", "best.pt"))
                
                img_size = 640
                
                start_val = time.time()
                metrics = model.val(data=data_yaml, imgsz=img_size, split="val", device=0, save=False)
                val_time = time.time() - start_val
                
                mAP50 = metrics.box.map50
                mAP5095 = metrics.box.map
                precision = metrics.box.mp
                recall = metrics.box.mr
                fps = 1000 / metrics.speed["inference"] if metrics.speed and "inference" in metrics.speed else 0
                
                TP = recall * 100
                FP = (1 - precision) * 100
                FN = (1 - recall) * 100
                TN = 100 - (TP + FP + FN)
                accuracy = max(0, min(1, (TP + TN) / 100))
                
                print(f"✅ {model_name}: mAP50={mAP50:.4f}, Acc={accuracy:.4f}")
                
                # Save to CSV
                for attempt in range(3):
                    try:
                        with open(csv_path, "a", newline="") as f:
                            writer = csv.writer(f)
                            writer.writerow([model_name, mAP50, mAP5095, precision, recall, accuracy, fps, 0, val_time])
                        print(f"📝 Results saved to CSV")
                        break
                    except PermissionError:
                        if attempt < 2:
                            print(f"⚠️ CSV locked. Retrying in 5 seconds...")
                            time.sleep(5)
                        else:
                            print(f"Results: {model_name},{mAP50},{mAP5095},{precision},{recall},{accuracy},{fps},0,{val_time}")
                
                completed_models.add(model_name)
                del model
                torch.cuda.empty_cache()
                gc.collect()
                continue
            else:
                print(f"📂 RESUMING from checkpoint: epoch {last_epoch + 1}/{epochs}")
                model = YOLO(checkpoint)
                resume = True
        except Exception as e:
            print(f"⚠️ Could not check checkpoint: {e}")
            print(f"🆕 STARTING FRESH")
            
            # Try both naming conventions for v11: yolov11n.pt and yolo11n.pt
            weights_path = os.path.join(weights_dir, f"{model_name}.pt")
            if not os.path.exists(weights_path) and "v11" in model_name:
                # Try without 'v' for YOLOv11 models
                alt_name = model_name.replace("yolov11", "yolo11")
                weights_path = os.path.join(weights_dir, f"{alt_name}.pt")
                print(f"🔍 Trying alternate name: {alt_name}.pt")
            
            if os.path.exists(weights_path):
                print(f"✅ Found local weights: {weights_path}")
                model = YOLO(weights_path)
            else:
                print(f"❌ Weights not found: {weights_path}")
                print(f"❌ Skipping {model_name}")
                continue
    else:
        print(f"🆕 STARTING FRESH")
        
        # Try both naming conventions for v11: yolov11n.pt and yolo11n.pt
        weights_path = os.path.join(weights_dir, f"{model_name}.pt")
        if not os.path.exists(weights_path) and "v11" in model_name:
            # Try without 'v' for YOLOv11 models
            alt_name = model_name.replace("yolov11", "yolo11")
            weights_path = os.path.join(weights_dir, f"{alt_name}.pt")
            print(f"🔍 Trying alternate name: {alt_name}.pt")
        
        if os.path.exists(weights_path):
            print(f"✅ Found local weights: {weights_path}")
            model = YOLO(weights_path)
        else:
            print(f"❌ Weights not found: {weights_path}")
            print(f"❌ Skipping {model_name}")
            continue
    
    start_train = time.time()
    
    # Optimized batch sizes for 10GB GPU
    if "x" in model_name:
        batch_size = 6
        img_size = 640
    elif "l" in model_name:
        batch_size = 10
        img_size = 640
    elif "m" in model_name:
        batch_size = 16
        img_size = 640
    elif "s" in model_name:
        batch_size = 24
        img_size = 640
    else:  # nano
        batch_size = 32
        img_size = 640
    
    print(f"📊 Settings: Batch={batch_size}, Image={img_size}, Cache=False, Workers=2")
    
    try:
        print(f"⏳ Starting training...")
        
        result = model.train(
            data=data_yaml,
            epochs=epochs,
            imgsz=img_size,
            device=0,
            project=output_dir,
            name=model_name,
            batch=batch_size,
            exist_ok=True,
            cache=False,
            workers=2,
            amp=True,
            plots=False,
            save_period=5,
            val=True,
            verbose=True,
            patience=20,
            resume=resume,
            close_mosaic=10
        )
        train_time = time.time() - start_train
        
        # Validate
        print(f"\n🔍 Evaluating {model_name}...")
        start_val = time.time()
        metrics = model.val(data=data_yaml, imgsz=img_size, split="val", device=0, save=False)
        val_time = time.time() - start_val
        
        mAP50 = metrics.box.map50
        mAP5095 = metrics.box.map
        precision = metrics.box.mp
        recall = metrics.box.mr
        fps = 1000 / metrics.speed["inference"] if metrics.speed and "inference" in metrics.speed else 0
        
        TP = recall * 100
        FP = (1 - precision) * 100
        FN = (1 - recall) * 100
        TN = 100 - (TP + FP + FN)
        accuracy = max(0, min(1, (TP + TN) / 100))
        
        print(f"\n✅ {model_name}: mAP50={mAP50:.4f}, Acc={accuracy:.4f}, Time={train_time/3600:.2f}h")
        
        # Save to CSV with retry logic
        max_retries = 3
        for attempt in range(max_retries):
            try:
                with open(csv_path, "a", newline="") as f:
                    writer = csv.writer(f)
                    writer.writerow([model_name, mAP50, mAP5095, precision, recall, accuracy, fps, train_time, val_time])
                print(f"📝 Results saved to CSV")
                break
            except PermissionError:
                if attempt < max_retries - 1:
                    print(f"⚠️ CSV file locked (close Excel?). Retrying in 5 seconds... ({attempt+1}/{max_retries})")
                    time.sleep(5)
                else:
                    print(f"❌ Could not save to CSV after {max_retries} attempts. Results:")
                    print(f"   {model_name},{mAP50},{mAP5095},{precision},{recall},{accuracy},{fps},{train_time},{val_time}")
        
        completed_models.add(model_name)
        print(f"🎉 {model_name} COMPLETE! ({len(completed_models)}/{len(all_models)})")
        
    except RuntimeError as e:
        if "out of memory" in str(e).lower():
            print(f"\n❌ OOM with batch={batch_size}. Retrying with batch={batch_size//2}...")
            torch.cuda.empty_cache()
            gc.collect()
            
            try:
                result = model.train(
                    data=data_yaml, epochs=epochs, imgsz=img_size, device=0,
                    project=output_dir, name=model_name, batch=batch_size//2,
                    exist_ok=True, cache=False, workers=2, amp=True,
                    plots=False, save_period=5, val=True, resume=resume
                )
                train_time = time.time() - start_train
                metrics = model.val(data=data_yaml, imgsz=img_size, split="val", device=0, save=False)
                
                mAP50 = metrics.box.map50
                mAP5095 = metrics.box.map
                precision = metrics.box.mp
                recall = metrics.box.mr
                fps = 1000 / metrics.speed["inference"] if metrics.speed and "inference" in metrics.speed else 0
                TP = recall * 100
                FP = (1 - precision) * 100
                FN = (1 - recall) * 100
                TN = 100 - (TP + FP + FN)
                accuracy = max(0, min(1, (TP + TN) / 100))
                
                for attempt in range(3):
                    try:
                        with open(csv_path, "a", newline="") as f:
                            writer = csv.writer(f)
                            writer.writerow([model_name, mAP50, mAP5095, precision, recall, accuracy, fps, train_time, val_time])
                        break
                    except PermissionError:
                        if attempt < 2:
                            time.sleep(5)
                        else:
                            print(f"Results: {model_name},{mAP50},{mAP5095},{precision},{recall},{accuracy},{fps},{train_time},{val_time}")
                
                print(f"✅ Retry succeeded!")
                completed_models.add(model_name)
            except Exception as e2:
                print(f"❌ Retry failed: {e2}")
                try:
                    with open(csv_path, "a", newline="") as f:
                        writer = csv.writer(f)
                        writer.writerow([model_name, "ERROR", "ERROR", "ERROR", "ERROR", "ERROR", "ERROR", 0, 0])
                except PermissionError:
                    print(f"Results: {model_name},ERROR,ERROR,ERROR,ERROR,ERROR,ERROR,0,0")
        else:
            print(f"❌ ERROR: {e}")
            try:
                with open(csv_path, "a", newline="") as f:
                    writer = csv.writer(f)
                    writer.writerow([model_name, "ERROR", "ERROR", "ERROR", "ERROR", "ERROR", "ERROR", 0, 0])
            except PermissionError:
                print(f"Results: {model_name},ERROR,ERROR,ERROR,ERROR,ERROR,ERROR,0,0")
    
    except Exception as e:
        print(f"❌ ERROR: {e}")
        import traceback
        traceback.print_exc()
        try:
            with open(csv_path, "a", newline="") as f:
                writer = csv.writer(f)
                writer.writerow([model_name, "ERROR", "ERROR", "ERROR", "ERROR", "ERROR", "ERROR", 0, 0])
        except PermissionError:
            print(f"⚠️ Could not write to CSV (file may be open)")
            print(f"Results: {model_name},ERROR,ERROR,ERROR,ERROR,ERROR,ERROR,0,0")
    
    finally:
        try:
            del model
        except:
            pass
        torch.cuda.empty_cache()
        gc.collect()
        
        if torch.cuda.is_available():
            mem = torch.cuda.memory_allocated(0) / 1024**3
            print(f"🧹 GPU Memory: {mem:.2f}GB\n")

# ================== FINAL SUMMARY ==================
print("\n" + "=" * 60)
print(f"🏁 ALL {len(all_models)} MODELS TRAINING COMPLETE!")
print("=" * 60)

# Show summary
try:
    import pandas as pd
    df = pd.read_csv(csv_path)
    df_clean = df[df['mAP50'] != 'ERROR'].copy()
    df_clean['mAP50'] = pd.to_numeric(df_clean['mAP50'])
    
    print(f"\n📊 FINAL RESULTS ({len(df_clean)}/{len(all_models)} successful):")
    print(df_clean.to_string(index=False))
    
    if len(df_clean) > 0:
        best_idx = df_clean['mAP50'].idxmax()
        print(f"\n🏆 BEST MODEL: {df_clean.loc[best_idx, 'Model']} - mAP50={df_clean.loc[best_idx, 'mAP50']:.4f}")
        
        # Show top 5
        print(f"\n🥇 TOP 5 MODELS:")
        top5 = df_clean.nlargest(5, 'mAP50')[['Model', 'mAP50', 'Accuracy', 'FPS']]
        for i, row in top5.iterrows():
            model_str = str(row['Model'])
            map_val = float(row['mAP50'])
            acc_val = float(row['Accuracy'])
            fps_val = float(row['FPS'])
            print(f"   {model_str:12} - mAP50: {map_val:.4f}, Acc: {acc_val:.4f}, FPS: {fps_val:.1f}")
    
    # Show any errors
    df_errors = df[df['mAP50'] == 'ERROR']
    if len(df_errors) > 0:
        print(f"\n⚠️ Models with errors: {', '.join(df_errors['Model'].tolist())}")
    
except PermissionError:
    print("⚠️ CSV file is locked (close Excel to view results)")
except Exception as e:
    print(f"⚠️ Could not display results: {e}")

print(f"\n📄 Results saved to: {csv_path}")
print(f"📦 Model weights: {output_dir}/*/weights/best.pt")


🚀 Training ALL 15 YOLO Models (v8, v10, v11)
📋 Total models to train: 15
   Models: yolov8n, yolov8s, yolov8m, yolov8l, yolov8x, yolov10n, yolov10s, yolov10m, yolov10l, yolov10x, yolov11n, yolov11s, yolov11m, yolov11l, yolov11x

✅ Found 10 completed models: {'yolov8x', 'yolov8n', 'yolov10x', 'yolov10s', 'yolov8s', 'yolov10m', 'yolov8l', 'yolov8m', 'yolov10n', 'yolov10l'}

📊 Progress: 10/15 completed, 5 remaining
   Remaining: yolov11n, yolov11s, yolov11m, yolov11l, yolov11x

⚡ Starting training loop...


[1/15] ⏩ Skipping yolov8n - already completed!

[2/15] ⏩ Skipping yolov8s - already completed!

[3/15] ⏩ Skipping yolov8m - already completed!

[4/15] ⏩ Skipping yolov8l - already completed!

[5/15] ⏩ Skipping yolov8x - already completed!

[6/15] ⏩ Skipping yolov10n - already completed!

[7/15] ⏩ Skipping yolov10s - already completed!

[8/15] ⏩ Skipping yolov10m - already completed!

[9/15] ⏩ Skipping yolov10l - already completed!

[10/15] ⏩ Skipping yolov10x - already completed!

[11/